# 📊 Entrainement des modèles et Prédiction des prix des voitures 

In [1]:
# Librairies


import os
import joblib
import pandas as pd
import numpy as np  
import seaborn as sns 
import plotly.express as px
import matplotlib.pyplot as plt

from pathlib import Path 

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score
from scipy.optimize import minimize_scalar
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


sns.set_theme(style="whitegrid", palette="muted")
from matplotlib import rcParams
palette = sns.color_palette("pastel")

import warnings
warnings.filterwarnings("ignore")

**Les données**

In [2]:
BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / 'Donnees'

#print(f"BASE_DIR: {BASE_DIR}")
#print(f"DATA_DIR: {DATA_DIR}")


df = pd.read_csv(DATA_DIR / 'vehicules.csv')
df.head(4)

,Modèle,Prix,Transmission,Version,Kilométrage,Carburant,Puissance,Évaluations,Vendeur,Nom de la Voiture,Date
0,audi,€ 74 999,Boîte automatique,55 TFSI Quattro Tiptronic S-Line INDIVIDUAL!,34 000 km,Essence,250 kW (340 CH),121,Kristof D'herde • BE-9300 Aalst,Audi Q855 TFSI Quattro Tiptronic S-Line INDIVI...,01/2021
1,audi,€ 118 900,Boîte automatique,60 Hybride 49 CO2 BlackPack B&O 23' Leather,10 km,Electrique/Essence,340 kW (462 CH),107,Frederik Rik Maxime Jorn Hendrik • BE-8710 Wie...,Audi Q860 Hybride 49 CO2 BlackPack B&O 23' Lea...,01/2024
2,audi,€ 118 900,Boîte automatique,60 Hybr 49gr Sline BlackPack B&O Leather 23',10 km,Electrique/Essence,340 kW (462 CH),107,Frederik Rik Maxime Jorn Hendrik • BE-8710 Wie...,Audi Q860 Hybr 49gr Sline BlackPack B&O Leathe...,01/2024
3,audi,€ 59 995,Boîte automatique,50 TDi S-Line Quattro Tiptronic,95 702 km,Diesel,210 kW (286 CH),75,Mhr Jannick Lazoore • BE-8620 Nieuwpoort,Audi Q850 TDi S-Line Quattro Tiptronic,04/2019


**Preprocessing des données**

In [3]:
from cleaning_data.cleaning import clean_preprocess
df = clean_preprocess(df)
df.head(4)

,prix,kilométrage,puissance,évaluations,modèle_Unknown,modèle_audi,modèle_bentley,modèle_ferrari,modèle_fiat,modèle_ford,...,modèle_renault,modèle_toyota,modèle_volkswagen,carburant_Diesel,carburant_Electrique,carburant_Electrique/Diesel,carburant_Electrique/Essence,carburant_Essence,transmission_Boîte automatique,transmission_Boîte manuelle
0,74999.0,34000.0,340.0,121.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,118900.0,10.0,462.0,107.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
2,118900.0,10.0,462.0,107.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
3,59995.0,95702.0,286.0,75.0,False,True,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False


In [10]:
#df.info()

dtypes: bool(20), float64(4)

**Variables numériques (4)**
- prix : float64
- kilométrage : float64
- puissance : float64
- évaluations : float64

**Variables catégorielles encodées(20)**
- Modèles (13 booléennes) :
modèle_audi, modèle_bentley, ..., modèle_volkswagen, etc.

modèle_Unknown : sûrement les véhicules où le modèle n’était pas identifié.

- Carburants (5 booléennes) :
Diesel, Essence, Electrique, ou des combinaisons (Electric/Essence, etc.)

- Transmission (2 booléennes) :
Boîte automatique / manuelle


In [7]:
print(f"la dimension de notre table  est : {df.shape}")

la dimension de notre table  est : (2670, 24)


In [8]:
print(f"Le nombre de valeurs manquantes : {df.isnull().sum().sum()}")

Le nombre de valeurs manquantes : 0


**Entrainement des modèles**

In [14]:
X = df.drop('prix', axis=1)
y = df['prix']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (2136, 23)
y_train shape: (2136,)
X_test shape: (534, 23)
y_test shape: (534,)
